# 19 - Trino: Federated Queries

Trino (dawniej PrestoSQL) - rozproszony silnik SQL do zapytań na danych z wielu źródeł.

**Tematy:**
- Architektura Trino: Coordinator, Worker, Connector
- Połączenie z Trino z Pythona (biblioteka `trino`)
- Konektory: PostgreSQL, Hive (HDFS/Parquet), Memory
- Podstawowe zapytania: SELECT, WHERE, GROUP BY, JOIN
- Federated queries - łączenie danych z wielu źródeł w jednym SQL
- Porównanie Trino vs Spark SQL
- EXPLAIN ANALYZE i cost-based optimizer
- Przeglądanie katalogów: SHOW CATALOGS, SHOW SCHEMAS, SHOW TABLES

## 1. Architektura Trino

```
                    ┌─────────────────────────────┐
                    │     Trino Coordinator        │
                    │                             │
                    │  - Parser / Planner         │
                    │  - Cost-Based Optimizer     │
                    │  - Scheduler                │
                    │  - REST API (:8080)         │
                    └──────────┬──────────────────┘
                               │
              ┌────────────────┼────────────────┐
              ▼                ▼                ▼
     ┌──────────────┐ ┌──────────────┐ ┌──────────────┐
     │ Trino Worker │ │ Trino Worker │ │ Trino Worker │
     │              │ │              │ │              │
     │  Execution   │ │  Execution   │ │  Execution   │
     │  Engine      │ │  Engine      │ │  Engine      │
     └──────┬───────┘ └──────┬───────┘ └──────┬───────┘
            │                │                │
     ┌──────┴───────┐ ┌─────┴────────┐ ┌─────┴────────┐
     │  Connector   │ │  Connector   │ │  Connector   │
     │  PostgreSQL  │ │  Hive/HDFS   │ │  Memory      │
     └──────────────┘ └──────────────┘ └──────────────┘
```

### Kluczowe elementy:

| Komponent | Rola | Odpowiednik w Spark |
|-----------|------|--------------------|
| **Coordinator** | Planuje i optymalizuje zapytania, rozdziela pracę | Driver |
| **Worker** | Wykonuje fragmenty zapytań (splits) | Executor |
| **Connector** | Adapter do źródła danych (wtyczka) | DataSource API |
| **Catalog** | Nazwany zestaw schematów (np. `postgresql`, `hive`) | Database |
| **Schema** | Zestaw tabel w katalogu | Schema |

### Czym Trino różni się od bazy danych?
- Trino **nie przechowuje danych** - tylko je odpytuje
- Dane zostają w swoim źródle (PostgreSQL, HDFS, S3, Kafka...)
- Trino = **compute engine** (jak Spark SQL, ale z naciskiem na interaktywne zapytania)

## 2. Setup - połączenie z Trino z Pythona

In [ ]:
!pip install trino pandas

In [ ]:
from trino.dbapi import connect
import pandas as pd

# Connection to Trino coordinator
# Trino uses catalogs to organize data sources
# Default catalog/schema can be set here, but we can also use fully qualified names
conn = connect(
    host="trino",
    port=8080,
    user="analyst",
    catalog="postgresql",
    schema="movielens"
)

def run_query(sql, conn=conn):
    """Execute a Trino query and return results as a pandas DataFrame."""
    cursor = conn.cursor()
    cursor.execute(sql)
    columns = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()
    return pd.DataFrame(rows, columns=columns)

def run_query_print(sql, conn=conn, max_rows=20):
    """Execute a Trino query, print results and return DataFrame."""
    df = run_query(sql, conn)
    print(f"Rows: {len(df)}")
    display(df.head(max_rows))
    return df

# Test connection
print("Connected to Trino!")
run_query_print("SELECT 'Hello from Trino!' AS message")

## 3. Konektory Trino

Trino używa **konektorów** (connectors) do łączenia się ze źródłami danych. Każdy konektor jest zarejestrowany jako **katalog** (catalog).

### Nasze katalogi:

| Catalog | Connector | Źródło danych | Opis |
|---------|-----------|--------------|------|
| `postgresql` | PostgreSQL Connector | PostgreSQL 16 | Ratings, movies - dane transakcyjne |
| `hive` | Hive Connector | HDFS (Parquet) | Dane analityczne, duże wolumeny |
| `memory` | Memory Connector | RAM Trino | Tymczasowe tabele, cache |
| `system` | System Connector | Trino metadata | Informacje o runtime, queries |

### Konfiguracja konektorów (pliki `.properties` w Trino):

```properties
# postgresql.properties
connector.name=postgresql
connection-url=jdbc:postgresql://postgres:5432/recommender
connection-user=recommender
connection-password=recommender

# hive.properties
connector.name=hive
hive.metastore.uri=thrift://hive-metastore:9083
hive.allow-drop-table=true

# memory.properties
connector.name=memory
memory.max-data-per-node=512MB
```

## 4. Przeglądanie katalogów i schematów

Trino ma wbudowane polecenia do eksploracji metadanych - analogicznie do `INFORMATION_SCHEMA` w PostgreSQL.

In [ ]:
# SHOW CATALOGS - lista wszystkich zarejestrowanych źródeł danych
run_query_print("SHOW CATALOGS")

In [ ]:
# SHOW SCHEMAS - lista schematów w danym katalogu
run_query_print("SHOW SCHEMAS FROM postgresql")

In [ ]:
# SHOW TABLES - lista tabel w danym schemacie
run_query_print("SHOW TABLES FROM postgresql.movielens")

In [ ]:
# DESCRIBE - struktura tabeli (kolumny, typy)
run_query_print("DESCRIBE postgresql.movielens.ratings")

In [ ]:
# SHOW COLUMNS - alternatywna składnia
run_query_print("SHOW COLUMNS FROM postgresql.movielens.movies")

In [ ]:
# Przeglądanie hive catalog
run_query_print("SHOW SCHEMAS FROM hive")

In [ ]:
# Tabele w hive
run_query_print("SHOW TABLES FROM hive.movielens")

In [ ]:
# Memory catalog - tymczasowe tabele
run_query_print("SHOW SCHEMAS FROM memory")

## 5. Podstawowe zapytania przez Trino

Składnia SQL w Trino jest zgodna z ANSI SQL. Główna różnica to **fully qualified names**: `catalog.schema.table`.

In [ ]:
# SELECT z WHERE - dane z PostgreSQL przez Trino
# Ponieważ conn ma ustawiony catalog=postgresql, schema=movielens,
# mozemy uzywac krotkich nazw
run_query_print("""
    SELECT user_id, movie_id, rating
    FROM ratings
    WHERE rating >= 4.5 AND user_id <= 100
    ORDER BY rating DESC
    LIMIT 10
""")

In [ ]:
# GROUP BY z HAVING
run_query_print("""
    SELECT user_id,
           COUNT(*) AS rating_count,
           ROUND(AVG(rating), 2) AS avg_rating,
           MIN(rating) AS min_rating,
           MAX(rating) AS max_rating
    FROM ratings
    GROUP BY user_id
    HAVING COUNT(*) > 1000
    ORDER BY rating_count DESC
    LIMIT 20
""")

In [ ]:
# JOIN w Trino - obie tabele z PostgreSQL
run_query_print("""
    SELECT m.title,
           COUNT(*) AS num_ratings,
           ROUND(AVG(r.rating), 2) AS avg_rating
    FROM ratings r
    JOIN movies m ON r.movie_id = m.movie_id
    GROUP BY m.title
    HAVING COUNT(*) > 5000
    ORDER BY avg_rating DESC
    LIMIT 20
""")

In [ ]:
# Fully qualified names - jawne wskazanie catalog.schema.table
# Przydatne gdy łączymy dane z wielu źródeł
run_query_print("""
    SELECT m.title, COUNT(*) AS cnt
    FROM postgresql.movielens.ratings r
    JOIN postgresql.movielens.movies m ON r.movie_id = m.movie_id
    WHERE r.rating = 5.0
    GROUP BY m.title
    ORDER BY cnt DESC
    LIMIT 10
""")

In [ ]:
# CTE (Common Table Expressions) - dziala identycznie jak w PostgreSQL i Spark SQL
run_query_print("""
    WITH user_stats AS (
        SELECT user_id,
               COUNT(*) AS num_ratings,
               ROUND(AVG(rating), 2) AS avg_rating
        FROM ratings
        GROUP BY user_id
    ),
    user_categories AS (
        SELECT *,
               CASE
                   WHEN num_ratings >= 1000 THEN 'power_user'
                   WHEN num_ratings >= 100 THEN 'active'
                   WHEN num_ratings >= 20 THEN 'casual'
                   ELSE 'rare'
               END AS user_category
        FROM user_stats
    )
    SELECT user_category,
           COUNT(*) AS num_users,
           ROUND(AVG(avg_rating), 2) AS mean_avg_rating,
           CAST(ROUND(AVG(num_ratings), 0) AS INTEGER) AS mean_num_ratings
    FROM user_categories
    GROUP BY user_category
    ORDER BY mean_num_ratings DESC
""")

## 6. Dane na HDFS (Hive Connector)

Hive Connector pozwala Trino odpytywać pliki na HDFS (Parquet, ORC, CSV, JSON) zarejestrowane w Hive Metastore.

### Jak dane trafiają do Hive?
1. Spark zapisuje Parquet na HDFS (notebook 15, 16)
2. Tabela jest zarejestrowana w Hive Metastore (CREATE TABLE / saveAsTable)
3. Trino widzi tabelę przez Hive Connector

In [ ]:
# Sprawdz co jest dostepne w hive catalog
run_query_print("SHOW SCHEMAS FROM hive")

In [ ]:
# Przyklady tabel hive (Parquet na HDFS)
run_query_print("SHOW TABLES FROM hive.movielens")

In [ ]:
# Odczyt danych z HDFS przez Trino
run_query_print("""
    SELECT *
    FROM hive.movielens.ratings
    LIMIT 10
""")

In [ ]:
# Analityka na danych HDFS - identyczny SQL!
run_query_print("""
    SELECT movie_id,
           COUNT(*) AS num_ratings,
           ROUND(AVG(rating), 2) AS avg_rating
    FROM hive.movielens.ratings
    GROUP BY movie_id
    HAVING COUNT(*) > 1000
    ORDER BY avg_rating DESC
    LIMIT 15
""")

In [ ]:
# Memory connector - tymczasowe tabele w RAM Trino
# Przydatne do cache'owania wynikow posrednich
cursor = conn.cursor()

# Utworz schema w memory
cursor.execute("CREATE SCHEMA IF NOT EXISTS memory.temp")

# Utworz tabele w pamieci z wynikiem zapytania
cursor.execute("""
    CREATE TABLE IF NOT EXISTS memory.temp.movie_stats AS
    SELECT movie_id,
           COUNT(*) AS num_ratings,
           AVG(rating) AS avg_rating,
           STDDEV(rating) AS std_rating
    FROM postgresql.movielens.ratings
    GROUP BY movie_id
""")

# Teraz tabela jest w pamięci - ultra szybki dostęp
run_query_print("""
    SELECT * FROM memory.temp.movie_stats
    ORDER BY num_ratings DESC
    LIMIT 10
""")

## 7. Federated Queries - serce Trino!

Najważniejsza cecha Trino: **jeden SQL łączy dane z wielu źródeł** - bez kopiowania danych!

```
┌──────────────────────────────────────────────────────┐
│                   Trino SQL Query                     │
│                                                       │
│  SELECT pg.title, hdfs.avg_rating, mem.user_segment  │
│  FROM postgresql.movielens.movies pg                  │
│  JOIN hive.movielens.ratings_agg hdfs                 │
│  JOIN memory.temp.user_segments mem                   │
│                                                       │
│     ┌──────────┐  ┌──────────┐  ┌──────────┐        │
│     │PostgreSQL │  │   HDFS   │  │  Memory  │        │
│     │  movies   │  │ ratings  │  │ segments │        │
│     └──────────┘  └──────────┘  └──────────┘        │
└──────────────────────────────────────────────────────┘
```

Trino:
1. Parsuje zapytanie
2. Rozpoznaje, które tabele są w których katalogach
3. Wysyła odpowiednie zapytania do każdego źródła (pushdown!)
4. Łączy wyniki w pamięci Workerów
5. Zwraca jeden wynik

In [ ]:
# FEDERATED QUERY: PostgreSQL + HDFS
# Metadata filmow z PostgreSQL + statystyki ocen z HDFS (Parquet)
run_query_print("""
    SELECT m.title,
           m.genres,
           h.num_ratings,
           ROUND(h.avg_rating, 2) AS avg_rating
    FROM postgresql.movielens.movies m
    JOIN (
        SELECT movie_id,
               COUNT(*) AS num_ratings,
               AVG(rating) AS avg_rating
        FROM hive.movielens.ratings
        GROUP BY movie_id
    ) h ON m.movie_id = h.movie_id
    WHERE h.num_ratings > 5000
    ORDER BY h.avg_rating DESC
    LIMIT 20
""")

In [ ]:
# FEDERATED QUERY: PostgreSQL + HDFS + Memory
# 3 źródła danych w jednym zapytaniu!
run_query_print("""
    WITH hdfs_user_stats AS (
        SELECT user_id,
               COUNT(*) AS num_ratings,
               AVG(rating) AS avg_rating
        FROM hive.movielens.ratings
        GROUP BY user_id
    ),
    top_users AS (
        SELECT user_id, num_ratings, avg_rating
        FROM hdfs_user_stats
        WHERE num_ratings > 500
    )
    SELECT tu.user_id,
           tu.num_ratings AS user_total_ratings,
           ROUND(tu.avg_rating, 2) AS user_avg_rating,
           m.title,
           r.rating AS user_movie_rating,
           ROUND(ms.avg_rating, 2) AS movie_global_avg
    FROM top_users tu
    JOIN postgresql.movielens.ratings r ON tu.user_id = r.user_id
    JOIN postgresql.movielens.movies m ON r.movie_id = m.movie_id
    JOIN memory.temp.movie_stats ms ON r.movie_id = ms.movie_id
    WHERE r.rating = 5.0
      AND ms.num_ratings > 10000
    ORDER BY tu.num_ratings DESC, m.title
    LIMIT 20
""")

In [ ]:
# Porównanie tych samych danych z różnych źródeł
# Weryfikacja spójności: PostgreSQL vs HDFS
run_query_print("""
    SELECT 'PostgreSQL' AS source,
           COUNT(*) AS total_ratings,
           COUNT(DISTINCT user_id) AS unique_users,
           COUNT(DISTINCT movie_id) AS unique_movies,
           ROUND(AVG(rating), 4) AS avg_rating
    FROM postgresql.movielens.ratings

    UNION ALL

    SELECT 'HDFS (Hive)' AS source,
           COUNT(*) AS total_ratings,
           COUNT(DISTINCT user_id) AS unique_users,
           COUNT(DISTINCT movie_id) AS unique_movies,
           ROUND(AVG(rating), 4) AS avg_rating
    FROM hive.movielens.ratings
""")

## 8. Trino vs Spark SQL - kiedy co użyć?

| Cecha | Trino | Spark SQL |
|-------|-------|-----------|
| **Główne zastosowanie** | Interaktywne zapytania, ad-hoc analytics | Batch processing, ETL, ML |
| **Latencja** | Niska (sekundy) | Wyższa (minuty) - narzut startu |
| **Przetwarzanie** | Pipeline (streaming między stages) | Stage-based (shuffle na dysk) |
| **Fault tolerance** | Brak - query restart przy awarii | Tak - retry na poziomie task |
| **Dane pośrednie** | W pamięci (szybko, ale limitowane) | Na dysku (wolniej, ale skalowalne) |
| **Federated queries** | Natywne - wiele katalogów | Możliwe, ale bardziej złożone |
| **UDF** | Ograniczone (Java/Trino plugins) | Bogata obsługa (Python, Scala) |
| **ML** | Brak wbudowanego | MLlib, ML Pipelines |
| **Ekosystem** | SQL-first | Programistyczny (Python/Scala + SQL) |

### Kiedy Trino?
- Szybkie, interaktywne zapytania analityczne (dashboardy, raporty)
- Odpytywanie wielu źródeł danych jednocześnie (federated queries)
- Ad-hoc analiza - analityk zadaje pytania bez pisania kodu
- Narzut startu Spark jest zbyt duży dla prostych zapytań
- BI tools (Tableau, Superset, Metabase) potrzebują szybkiego SQL endpoint

### Kiedy Spark SQL?
- Duże transformacje ETL (terabajty danych)
- Machine learning (MLlib, integration z Python ML ecosystem)
- Złożone pipeline'y wymagające fault tolerance
- Streaming (Structured Streaming)
- Gdy potrzebna jest programistyczna kontrola (DataFrames, UDFs w Pythonie)

### W praktyce: razem!
```
Spark (batch ETL) → HDFS/S3 (Parquet) → Trino (interaktywne zapytania)
```
Spark przetwarza i zapisuje dane, Trino odpytuje je interaktywnie.

In [ ]:
# Benchmark: Trino query latency
import time

queries = {
    "simple_count": "SELECT COUNT(*) FROM postgresql.movielens.ratings",
    "group_by": """
        SELECT movie_id, COUNT(*) AS cnt, AVG(rating) AS avg_r
        FROM postgresql.movielens.ratings
        GROUP BY movie_id
        ORDER BY cnt DESC
        LIMIT 10
    """,
    "join_query": """
        SELECT m.title, COUNT(*) AS cnt
        FROM postgresql.movielens.ratings r
        JOIN postgresql.movielens.movies m ON r.movie_id = m.movie_id
        WHERE r.rating >= 4.0
        GROUP BY m.title
        ORDER BY cnt DESC
        LIMIT 10
    """,
    "federated": """
        SELECT m.title, h.avg_rating
        FROM postgresql.movielens.movies m
        JOIN (
            SELECT movie_id, AVG(rating) AS avg_rating
            FROM hive.movielens.ratings
            GROUP BY movie_id
            HAVING COUNT(*) > 1000
        ) h ON m.movie_id = h.movie_id
        ORDER BY h.avg_rating DESC
        LIMIT 10
    """
}

for name, sql in queries.items():
    start = time.time()
    run_query(sql)
    elapsed = time.time() - start
    print(f"{name:<20} {elapsed:.2f}s")

## 9. EXPLAIN ANALYZE - Cost-Based Optimizer

Trino ma zaawansowany **Cost-Based Optimizer (CBO)**, który:
- Zbiera statystyki o tabelach (rozmiar, cardinalność, rozkład wartości)
- Wybiera optymalną kolejność joinów
- Decyduje o strategii join (broadcast vs distributed)
- Wykonuje predicate pushdown do źródeł danych

### EXPLAIN vs EXPLAIN ANALYZE:
- `EXPLAIN` - pokazuje plan zapytania **bez wykonania**
- `EXPLAIN ANALYZE` - wykonuje zapytanie i pokazuje **rzeczywiste** koszty

In [ ]:
# EXPLAIN - plan logiczny
explain_result = run_query("""
    EXPLAIN
    SELECT m.title, COUNT(*) AS num_ratings, AVG(r.rating) AS avg_rating
    FROM postgresql.movielens.ratings r
    JOIN postgresql.movielens.movies m ON r.movie_id = m.movie_id
    GROUP BY m.title
    ORDER BY num_ratings DESC
    LIMIT 10
""")

# Plan jest w jednej kolumnie - wypisz czytelnie
for _, row in explain_result.iterrows():
    print(row.iloc[0])

In [ ]:
# EXPLAIN ANALYZE - rzeczywiste wykonanie z metrykami
explain_analyze = run_query("""
    EXPLAIN ANALYZE
    SELECT m.title, COUNT(*) AS num_ratings, AVG(r.rating) AS avg_rating
    FROM postgresql.movielens.ratings r
    JOIN postgresql.movielens.movies m ON r.movie_id = m.movie_id
    GROUP BY m.title
    HAVING COUNT(*) > 1000
    ORDER BY avg_rating DESC
    LIMIT 10
""")

for _, row in explain_analyze.iterrows():
    print(row.iloc[0])

In [ ]:
# EXPLAIN z roznym typem wyjscia
# EXPLAIN (TYPE DISTRIBUTED) - plan z uwzglednieniem partycjonowania
distributed_plan = run_query("""
    EXPLAIN (TYPE DISTRIBUTED)
    SELECT m.title, COUNT(*) AS num_ratings
    FROM postgresql.movielens.ratings r
    JOIN postgresql.movielens.movies m ON r.movie_id = m.movie_id
    GROUP BY m.title
    ORDER BY num_ratings DESC
    LIMIT 10
""")

for _, row in distributed_plan.iterrows():
    print(row.iloc[0])

In [ ]:
# EXPLAIN na federated query - zobaczmy jak Trino rozkłada pracę miedzy źródła
federated_plan = run_query("""
    EXPLAIN ANALYZE
    SELECT m.title,
           pg_r.pg_count,
           hdfs_r.hdfs_count
    FROM postgresql.movielens.movies m
    JOIN (
        SELECT movie_id, COUNT(*) AS pg_count
        FROM postgresql.movielens.ratings
        GROUP BY movie_id
    ) pg_r ON m.movie_id = pg_r.movie_id
    JOIN (
        SELECT movie_id, COUNT(*) AS hdfs_count
        FROM hive.movielens.ratings
        GROUP BY movie_id
    ) hdfs_r ON m.movie_id = hdfs_r.movie_id
    ORDER BY pg_r.pg_count DESC
    LIMIT 10
""")

for _, row in federated_plan.iterrows():
    print(row.iloc[0])

### Co warto obserwować w EXPLAIN ANALYZE:

| Metryka | Opis | Na co zwrócić uwagę |
|---------|------|---------------------|
| **Rows** | Liczba przetworzonych wierszy | Czy optimizer dobrze estymuje? |
| **CPU time** | Czas CPU na danym etapie | Gdzie bottleneck? |
| **Wall time** | Rzeczywisty czas | Czy czekamy na I/O? |
| **Connector pushdown** | Filtr przesunięty do źródła | Mniej danych transferowanych |
| **Join strategy** | HASH vs BROADCAST | Broadcast dla małych tabel |

## 10. Zaawansowane federated queries

Kilka praktycznych wzorców użycia federated queries.

In [ ]:
# Wzorzec 1: Enrichment - wzbogacenie danych z HDFS metadanymi z PostgreSQL
run_query_print("""
    WITH hdfs_ratings AS (
        SELECT movie_id,
               user_id,
               rating,
               rating_timestamp
        FROM hive.movielens.ratings
        WHERE rating >= 4.5
    )
    SELECT m.title,
           m.genres,
           COUNT(*) AS high_ratings,
           MIN(hr.rating_timestamp) AS first_high_rating,
           MAX(hr.rating_timestamp) AS last_high_rating
    FROM hdfs_ratings hr
    JOIN postgresql.movielens.movies m ON hr.movie_id = m.movie_id
    GROUP BY m.title, m.genres
    ORDER BY high_ratings DESC
    LIMIT 15
""")

In [ ]:
# Wzorzec 2: Cross-source aggregation
# Porownaj wyniki agregacji z PostgreSQL i HDFS per movie
run_query_print("""
    WITH pg_stats AS (
        SELECT movie_id,
               COUNT(*) AS pg_count,
               ROUND(AVG(rating), 3) AS pg_avg
        FROM postgresql.movielens.ratings
        GROUP BY movie_id
    ),
    hdfs_stats AS (
        SELECT movie_id,
               COUNT(*) AS hdfs_count,
               ROUND(AVG(rating), 3) AS hdfs_avg
        FROM hive.movielens.ratings
        GROUP BY movie_id
    )
    SELECT m.title,
           pg.pg_count,
           h.hdfs_count,
           pg.pg_avg,
           h.hdfs_avg,
           ABS(pg.pg_avg - h.hdfs_avg) AS avg_diff
    FROM pg_stats pg
    JOIN hdfs_stats h ON pg.movie_id = h.movie_id
    JOIN postgresql.movielens.movies m ON pg.movie_id = m.movie_id
    WHERE pg.pg_count > 1000
    ORDER BY avg_diff DESC
    LIMIT 15
""")

In [ ]:
# Wzorzec 3: CREATE TABLE AS SELECT (CTAS) - materializacja wynikow
# Zapisz wynik federated query do memory lub hive
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS memory.temp.movie_enriched AS
    SELECT m.movie_id,
           m.title,
           m.genres,
           COALESCE(r.num_ratings, 0) AS num_ratings,
           COALESCE(r.avg_rating, 0) AS avg_rating,
           COALESCE(r.std_rating, 0) AS std_rating
    FROM postgresql.movielens.movies m
    LEFT JOIN memory.temp.movie_stats r ON m.movie_id = r.movie_id
""")

run_query_print("""
    SELECT * FROM memory.temp.movie_enriched
    WHERE num_ratings > 10000
    ORDER BY avg_rating DESC
    LIMIT 10
""")

## 11. System Connector - monitorowanie Trino

Trino ma wbudowany `system` catalog z informacjami o runtime.

In [ ]:
# Informacje o wersji i wezlach
run_query_print("SELECT * FROM system.runtime.nodes")

In [ ]:
# Aktualnie wykonywane zapytania
run_query_print("""
    SELECT query_id, state, query, started
    FROM system.runtime.queries
    ORDER BY started DESC
    LIMIT 10
""")

In [ ]:
# Informacje o taskach
run_query_print("""
    SELECT node_id,
           COUNT(*) AS num_tasks,
           SUM(input_rows) AS total_input_rows,
           SUM(output_rows) AS total_output_rows
    FROM system.runtime.tasks
    GROUP BY node_id
""")

## Zadanie 1

Napisz **federated query**, które łączy dane z **3 źródeł** (PostgreSQL, Hive/HDFS, Memory):

1. Z `postgresql.movielens.movies` - weź tytuły i gatunki filmów
2. Z `hive.movielens.ratings` - oblicz statystyki ocen per film (count, avg, min, max)
3. Z `memory.temp.movie_stats` (utworzony wcześniej) - weź odchylenie standardowe

Wynik powinien zawierać:
- Tytuł filmu, gatunek
- Liczbę ocen, średnią, min, max, odchylenie standardowe
- Tylko filmy z > 5000 ocen
- Posortowane po odchyleniu standardowym malejąco (najbardziej "kontrowersyjne" filmy)

In [ ]:
# Twoje rozwiązanie:
run_query_print("""

""")

## Zadanie końcowe

Zbuduj **analytyczny dashboard query** łączący dane z PostgreSQL + HDFS:

Stwórz raport, który w jednym zapytaniu (federated!) pokaże:

1. **Top 10 filmów** - z PostgreSQL (`movies`) + HDFS (`ratings`):
   - Tytuł, gatunek, liczba ocen, średnia ocena
   - Filtruj: minimum 10000 ocen

2. **Segmentacja użytkowników** - z HDFS (`ratings`):
   - Power users (>1000 ocen), Active (100-1000), Casual (20-100), Rare (<20)
   - Dla każdego segmentu: liczba użytkowników, średnia ocena, średnia liczba ocen

3. **Trend popularności** - z HDFS (`ratings`) + PostgreSQL (`movies`):
   - Wyciągnij rok z tytułu filmu (np. "Toy Story (1995)" → 1995)
   - Ile ocen zebrały filmy per dekada (1990s, 2000s, 2010s)
   - Średnia ocena per dekada

Zapisz wynik każdej części do `memory.temp` jako osobną tabelę, a na końcu połącz w summary.

In [ ]:
# Twoje rozwiązanie:

# Część 1: Top 10 filmów
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS memory.temp.dashboard_top_movies AS
    SELECT 1 AS placeholder
    -- Twoje zapytanie tutaj
""")

run_query_print("SELECT * FROM memory.temp.dashboard_top_movies")

In [ ]:
# Część 2: Segmentacja użytkowników


In [ ]:
# Część 3: Trend popularności per dekada


In [ ]:
# Podsumowanie: polacz wszystkie czesci


In [ ]:
# Cleanup memory tables
cursor = conn.cursor()
for table in ['movie_stats', 'movie_enriched', 'dashboard_top_movies']:
    try:
        cursor.execute(f"DROP TABLE IF EXISTS memory.temp.{table}")
    except Exception as e:
        print(f"Could not drop {table}: {e}")

conn.close()
print("Done!")